<a href="https://colab.research.google.com/github/jaderAdriel/calculo-ndwi/blob/main/ndwi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geemap
!pip install earthengine-api
!pip install rasterio
!pip install earthpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.5 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=f0bebde3fce6ac26423b4bfda0114bf428cd6f816f4164d6f19f067

In [ ]:
#importando bibliotecas
import ee 
import geemap
import requests
import numpy as np
import matplotlib.pyplot as plt
import io
import zipfile
import rasterio
import earthpy.plot as ep

In [ ]:
#autenticando e incializando
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=g2RPRhsGU-bH_ODmgabjWLtXxeoG0AmCvpPHFdvR9lo&tc=V83o8wn7hSCy2vhcMVH5WuixQXvSZT_Ydgh0b3lU6YI&cc=VIrvMdyY32hej8Tm2JoOYMtYCujjGSIEYL3vI7Dpujg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VNyoEv6ZkGZ_bZvnkBW_eJ9L0jAmh1-nnIfrTRAE1puez5iAjM8eLc

Successfully saved authorization token.


In [ ]:
def getDownloadUrlForSentinelBands(polygonCoords, bands, scale=10, start_date='2022-01-01', end_date='2022-04-30', CLOUDY_PIXEL_PERCENTAGE=20 ):
    roi = ee.Geometry.Polygon(polygonCoords)

    date_range = ee.DateRange(start_date, end_date)

    collection = ee.ImageCollection('COPERNICUS/S2_SR') \
                    .filterBounds(roi) \
                    .filterDate(date_range) \
                    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', CLOUDY_PIXEL_PERCENTAGE) \
                    .sort('CLOUDY_PIXEL_PERCENTAGE') 

    image = ee.Image(collection.first()).select(bands)

    # Obtém a URL de download
    url = image.getDownloadURL({'region': roi.getInfo()['coordinates'], 'scale': scale})

    return url


In [ ]:
def getBandsFromZipFile(zip):
    bands = {}

    with zipfile.ZipFile(io.BytesIO(zip)) as zip_file:
        for file_name in zip_file.namelist():
            with zip_file.open(file_name) as file:
                with rasterio.open(file) as src:
                    
                    band_name = file_name[-5:-4] 
                    bands[band_name] = src.read(1)

    return bands

In [ ]:
def calcNDWIFromPolygonCoords(polygonCoords, start_date='2022-01-01', end_date='2022-04-30'):
    
    # pega a url de download para bandas de uma imagem do sentinel no póligono dado
    downloadBandsUrl = getDownloadUrlForSentinelBands(polygonCoords, ['B4', 'B8'], start_date=start_date, end_date=end_date)
    
    # faz o download das bandas pela a url fornecida
    zippedBands = requests.get(downloadBandsUrl).content

    # extrai o arquivo zippado, e pega as bandas respectivas de cada arquivo extraido
    bandsMap = getBandsFromZipFile(zippedBands)

    np.seterr(divide='ignore', invalid='ignore')

    nir, green = [ bandsMap.get("4"), bandsMap.get("8") ]

    # calculo do ndwi
    ndwi = (green - nir) / (green + nir)

    return ndwi


In [ ]:
def plotNDWI(ndwi, cmap):
    ep.plot_bands(ndwi,
              cmap=cmap,
              scale=False,
              vmin=0, vmax=1,
            )
    plt.show()


In [ ]:
#@title NDWI EXEMPLO


#coordenadas do poligono dE EXEMPLO
coordsPolyA = [
    [-45.2362703741911,-18.946652390194988],
    [-45.04126304997235,-18.946652390194988],
    [-45.04126304997235,-18.807613641159648],
    [-45.2362703741911,-18.807613641159648],
    [-45.2362703741911,-18.946652390194988],
]
# dia 
# ToDo:
# 1. integrar shapeFiles para ler diferentes poligonos.
# 2. Pegar ndwi de 6 reservatorios diferentes: uruçuca - ilheus, ceraima e outras distribuidas
# gedal, fiona, ogr -> shapefile

ndwiA1 = calcNDWIFromPolygonCoords(coordsPolyA, start_date='2022-01-01', end_date='2022-02-27')
ndwiA2 = calcNDWIFromPolygonCoords(coordsPolyA, start_date='2022-08-01', end_date='2022-10-30')
plotNDWI(ndwiA1, 'RdYlBu')
plotNDWI(ndwiA2, 'RdYlBu')

In [ ]:
#@title NDWI BARRAGEM DE CERAIMA


#coordenadas do poligono da barragem de ceraima
coordsPolyB = [[-42.70026304095808,-14.29815179793395],
    [-42.649279612003,-14.29815179793395],
    [-42.649279612003,-14.256062844852245],
    [-42.70026304095808,-14.256062844852245],
    [-42.70026304095808,-14.29815179793395]
]


ndwiB1 = calcNDWIFromPolygonCoords(coordsPolyB, start_date='2022-01-01', end_date='2022-04-27')
ndwiB2 = calcNDWIFromPolygonCoords(coordsPolyB, start_date='2022-08-01', end_date='2022-10-30')
plotNDWI(ndwiB1, 'RdYlBu')
plotNDWI(ndwiB2, 'RdYlBu')
